In [ ]:
import cv2
import numpy as np
import time
import pygame
import smtplib
from tensorflow.keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications import EfficientNetB0
#from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
Alarm_Status = False
Email_Status = False

In [ ]:
# Initialize Pygame
pygame.mixer.pre_init(44100, -16, 2, 512)
pygame.init()

In [ ]:
# Add custom layers for fire detection
model = load_model('resnetmodel.h5')

In [ ]:
# Load the alarm sound file
alarm_sound = pygame.mixer.Sound("Alarm.mp3")

In [ ]:
#connection established for mail
conn=smtplib.SMTP('smtp.gmail.com',587)

In [ ]:
type(conn)

In [ ]:
conn.ehlo()
conn.starttls()

In [ ]:
#login for admin
conn.login('dipenthapachhetri10@gmail.com','wmgv gyii wghb wbup')

In [ ]:
# Open the camera (0 represents the default camera)
cap = cv2.VideoCapture(0)

In [ ]:
# Define a function for real-time fire detection
def detect_fire(frame):
    # Preprocess the frame
    frame = cv2.resize(frame, (224, 224))
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for efficientnet only
    frame = image.img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    frame = preprocess_input(frame)

    # Use the model for prediction
    prediction = model.predict(frame)
    
    # Assuming binary classification (fire or no fire)
    if prediction[0][0] > 0.5:
        return True
    else:
        return False


In [ ]:
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Perform fire detection
    is_fire = detect_fire(frame)

    # Display the original frame
    cv2.imshow('Fire Detection', frame)

    # If the prediction indicates fire
    if is_fire:
        print("Fire detected!")
        pygame.mixer.Channel(0).play(alarm_sound, loops=-1)
        alarm_playing = True
        start_time = time.time()  # Record the start time
        email_sending = True
        conn.sendmail('dipenthapachhetri10@gmail.com','chhanprakash@gmail.com','Subject: ALERT!!!!!!! \n \n  There is huge fire.')
    
    elif not is_fire:
        print("No fire")
        pygame.mixer.Channel(0).stop()
        alarm_playing = False
        email_sending = False

        
     # Check if the alarm has been playing for more than 11 seconds
    if alarm_playing and (time.time() - start_time > 11):
        pygame.mixer.Channel(0).stop()
        alarm_playing = False
        email_sending = False
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
